In [ ]:
pip install rioxarray

In [ ]:
pip install xarray

In [ ]:
pip install earthengine-api rasterio

In [ ]:
pip install geopandas

In [ ]:
pip install gdal==$(gdal-config --version) geemap

In [ ]:
import ee
import geopandas as gpd
import os
from datetime import datetime, timedelta
import xarray as xr
import rioxarray
import rasterio

from rasterio.enums import Resampling
import numpy as np
from collections import defaultdict

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='btech-project-443813') # Make sure this project ID is correct

In [ ]:
pip install setuptools geemap --upgrade

In [ ]:
import ee
import geemap
import os
from datetime import datetime, timedelta

# Initialize Earth Engine
ee.Initialize()

# Define the region of interest (ROI)
# Define the region of interest (ROI) for Delhi, India
roi = ee.Geometry.Polygon([
    [76.8, 28.9],  # Northwest point
    [77.6, 28.9],  # Northeast point
    [77.6, 28.4],  # Southeast point
    [76.8, 28.4]   # Southwest point
])

# Define the date range
start_date = datetime(2008, 10, 1)
end_date = datetime(2023, 3, 31)

# Define the output directory on your local machine
output_dir = r'/home/stormej/dev/btech-project/data/ndvi/ndvi_tif'
os.makedirs(output_dir, exist_ok=True)


# Function to get NDVI monthly data

def get_monthly_ndvi(year, month):
    start = datetime(year, month, 1)
    end = (start + timedelta(days=32)).replace(day=1)

    dataset = ee.ImageCollection('MODIS/006/MOD09GA') \
        .filterDate(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    
    def addNDVI(image):
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
        return image.addBands(ndvi)
    
    monthly_ndvi = dataset.map(addNDVI).select('NDVI').mean().clip(roi)
    return monthly_ndvi

# Function to download NDVI for a monthly data

def download_monthly_ndvi(year, month):
    date_str = datetime(year, month, 1).strftime('%Y-%m')
    ndvi = get_monthly_ndvi(year, month)
    filename = os.path.join(output_dir, f'ndvi_{date_str}.tif')

    geemap.ee_export_image(
        ndvi, 
        filename=filename, 
        scale=500,
        region=roi.getInfo()['coordinates'],
        crs='EPSG:4326'
    )
    print(f"Downloaded monthly NDVI for {date_str}")

# Download NDVI data for each day in the date range
current_date = start_date
while current_date <= end_date:
    download_monthly_ndvi(current_date.year, current_date.month)
    current_date += timedelta(days=1)

print("All NDVI data downloaded.")

/home/stormej/dev/btech-project/env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09GA! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09GA

  warnings.warn(warning, category=DeprecationWarning)


Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-project/data/ndvi_tif/ndvi_2008-10.tif
Downloaded monthly NDVI for 2008-10
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/btech-proj

In [ ]:
#Resampling ndvi monthly data to imd data i.e 0.25 degree

import os
import rasterio
from rasterio.enums import Resampling
# # Paths
# rainfall_folder = r'/home/stormej/dev/btech-project/data/rain_tif'  # Folder containing all rainfall files
rainfall_folder = r'/home/stormej/dev/btech-project/data/reference_tif'  # Folder containing reference files
input_folder = r'/home/stormej/dev/btech-project/data/ndvi/ndvi_tif'  # Folder containing all LST files
output_folder = r'/home/stormej/dev/btech-project/data/ndvi/ndvi_resampled_monthly'  # Folder to save resampled files

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process all LST files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        lst_path = os.path.join(input_folder, filename)
        print(f"\nProcessing: {filename}")
        
        with rasterio.open(lst_path) as lst_src:
            lst_data = lst_src.read(1)
            print(f"LST data shape: {lst_data.shape}, dtype: {lst_data.dtype}")
            
            # Resample LST data using average method
            resampled_lst = lst_src.read(
                out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
                resampling=Resampling.average
            )
            print(f"Resampled LST shape: {resampled_lst.shape}, dtype: {resampled_lst.dtype}")
            
            # Extract date from LST filename (handling format: lst_south_2010-01-01.tif)
            # Split by underscore and get the last part before .tif
            date_part = filename.split('_')[-1].split('.')[0]
            
            # Create output filename
            output_filename = f'modis_ndvi_avg_resampled_{date_part}.tif'
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the resampled LST
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=resampled_lst.shape[1],
                width=resampled_lst.shape[2],
                count=1,
                dtype=resampled_lst.dtype,
                crs=rainfall_crs,
                transform=rainfall_transform,
            ) as dst:
                dst.write(resampled_lst[0], 1)
        
        print(f"Resampled LST saved to: {output_path}")

print("\nAll LST files have been resampled.")

Using reference rainfall file: /home/stormej/dev/btech-project/data/reference_tif/refrence_tif.tif
Reference rainfall data shape: (129, 135), CRS: EPSG:4326

Processing: ndvi_2000-02.tif
LST data shape: (113, 179), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/ndvi_resampled_monthly/modis_ndvi_avg_resampled_2000-02.tif

Processing: ndvi_2000-03.tif
LST data shape: (113, 179), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/ndvi_resampled_monthly/modis_ndvi_avg_resampled_2000-03.tif

Processing: ndvi_2000-04.tif
LST data shape: (113, 179), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/ndvi_resampled_monthly/modis_ndvi_avg_resampled_2000-04.tif

Processing: ndvi_2000-05.tif
LST data shape: (113, 179), dtype: float32
Resampled LST shape: (1, 129, 135),